In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime
import time
import re

In [ ]:
file_path = '/content/drive/MyDrive/Merged Data/LatestData/Master2.xlsx'
df = pd.read_excel(file_path, na_values='--')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
df.columns

Index(['Scheme_name', 'Category', 'Category_Equity', 'Category_Debt',
       'Category_Hybrid', 'Expense_ratio', 'Risk_factor', 'age_in_months',
       'Rating', 'Launch', 'Net_assets', 'Return_1wk', 'Return_1m',
       'Return_3m', 'Return_6m', 'Return_1yr', 'Return_3yr', 'Return_5yr',
       'Return_10yr', 'Return_SIP_3yr', 'Return_SIP_5yr', 'Return_SIP_10yr',
       'Equity Fund Style', 'Equity_fund_style_Growth',
       'Equity_fund_style_Value', 'Equity_fund_style_Blend', 'YTM',
       'Standard Deviation', 'Sharpe Ratio', 'Sortino Ratio', 'Beta', 'Alpha',
       'R-Squared', 'NAV_latest', 'NAV_previous', 'NAV_52wk_high',
       'NAV_52wk_low', 'Minimum_Investment', 'Fund_Manager_Tenure',
       'Fund_manager'],
      dtype='object')

In [ ]:
df.shape

(926, 40)

In [ ]:
#Selecting funds >18 months
# print(type(df['age_in_months'].astype('int')))
df = df[df['age_in_months']>18]
df.shape

(926, 40)

In [ ]:
#Drop rows with no return values
df = df.dropna(subset=['Return_1wk','Return_1m','Return_3m'], how='all')
df.shape

(926, 40)

In [ ]:
#Create funds managed by fund manager column
funds_managed_by = df['Fund_manager'].value_counts().to_dict()
df['#_Funds_managed'] = df.apply(lambda row: funds_managed_by[row.Fund_manager], axis=1)
# df = df.replace({'Fund_manager': funds_managed_by})


In [ ]:
df.head

<bound method NDFrame.head of                                            Scheme_name  Category  \
0    Aditya Birla Sun Life Bal Bhavishya Yojna - Di...    EQ-FLX   
1    Aditya Birla Sun Life Banking & Financial Serv...   EQ-BANK   
2    Aditya Birla Sun Life Commodity Equities Fund ...   EQ-INTL   
3    Aditya Birla Sun Life Digital India Fund - Dir...     EQ-IT   
4    Aditya Birla Sun Life Dividend Yield Fund - Di...  EQ-DIV Y   
..                                                 ...       ...   
921               UTI Hybrid Equity Fund - Direct Plan     HY-AH   
922                 UTI Multi Asset Fund - Direct Plan    HY-MAA   
923             UTI Regular Savings Fund - Direct Plan     HY-CH   
924  UTI Retirement Benefit Pension Fund - Direct Plan     HY-BH   
925       UTI Unit Linked Insurance Plan - Direct Plan    HY-DAA   

     Category_Equity  Category_Debt  Category_Hybrid  Expense_ratio  \
0                  1              0                0           0.76   
1          

In [ ]:
#Replacing NA values w column average
df['Rating'].sum()/1080
#avg rating is 2

df['Rating'] = df['Rating'].fillna(2)
df.head


<bound method NDFrame.head of                                            Scheme_name  Category  \
0    Aditya Birla Sun Life Bal Bhavishya Yojna - Di...    EQ-FLX   
1    Aditya Birla Sun Life Banking & Financial Serv...   EQ-BANK   
2    Aditya Birla Sun Life Commodity Equities Fund ...   EQ-INTL   
3    Aditya Birla Sun Life Digital India Fund - Dir...     EQ-IT   
4    Aditya Birla Sun Life Dividend Yield Fund - Di...  EQ-DIV Y   
..                                                 ...       ...   
921               UTI Hybrid Equity Fund - Direct Plan     HY-AH   
922                 UTI Multi Asset Fund - Direct Plan    HY-MAA   
923             UTI Regular Savings Fund - Direct Plan     HY-CH   
924  UTI Retirement Benefit Pension Fund - Direct Plan     HY-BH   
925       UTI Unit Linked Insurance Plan - Direct Plan    HY-DAA   

     Category_Equity  Category_Debt  Category_Hybrid  Expense_ratio  \
0                  1              0                0           0.76   
1          

In [ ]:
# Update NA return values with 0 and drop columns
# Drop NAV_previous since correlation with NAV_latest=1
df.update(df[['Return_1wk','Return_1m', 'Return_3m', 'Return_6m', 'Return_1yr', 'Return_3yr', 'Return_5yr', 'Return_10yr']].fillna(0))
df.drop(['YTM','Beta', 'Alpha','R-Squared', 'Equity Fund Style', 'Launch', 'Return_SIP_3yr', 'Return_SIP_5yr', 'Return_SIP_10yr', 'NAV_previous'], axis=1, inplace=True)
  									

In [ ]:
df['Risk_factor'].value_counts()

Very High          530
Moderate           146
Low to Moderate    106
Moderately High     64
Low                 59
High                21
Name: Risk_factor, dtype: int64

In [ ]:
risk_factor_map = {'Very High':5, 'Moderate':2, 'Low to Moderate':1, 'Moderately High':3, 'Low':0, 'High':4}
df['Risk_factor'] = df['Risk_factor'].map(risk_factor_map)
df.head

<bound method NDFrame.head of                                            Scheme_name  Category  \
0    Aditya Birla Sun Life Bal Bhavishya Yojna - Di...    EQ-FLX   
1    Aditya Birla Sun Life Banking & Financial Serv...   EQ-BANK   
2    Aditya Birla Sun Life Commodity Equities Fund ...   EQ-INTL   
3    Aditya Birla Sun Life Digital India Fund - Dir...     EQ-IT   
4    Aditya Birla Sun Life Dividend Yield Fund - Di...  EQ-DIV Y   
..                                                 ...       ...   
921               UTI Hybrid Equity Fund - Direct Plan     HY-AH   
922                 UTI Multi Asset Fund - Direct Plan    HY-MAA   
923             UTI Regular Savings Fund - Direct Plan     HY-CH   
924  UTI Retirement Benefit Pension Fund - Direct Plan     HY-BH   
925       UTI Unit Linked Insurance Plan - Direct Plan    HY-DAA   

     Category_Equity  Category_Debt  Category_Hybrid  Expense_ratio  \
0                  1              0                0           0.76   
1          

In [ ]:
df.shape
df.to_excel('/content/drive/MyDrive/Merged Data/LatestData/ProcessedData3.xlsx', index=False)